In [1]:
from compress_llama_with_kronsvd_llama2 import evaluate_model

In [3]:
import json
import logging
import math
import os
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
from datasets import load_dataset
from safetensors.torch import load_file
from torch.utils.data.dataset import Dataset
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
model = AutoModelForCausalLM.from_pretrained("/home/jovyan/shares/SR004.nfs2/chekalina/FisherKronecker/llama/llama10", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("/home/jovyan/shares/SR004.nfs2/chekalina/FisherKronecker/llama/llama10")

[2025-04-24 22:24:06,655] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2025-04-24 22:24:06,871 - INFO - /home/jovyan/.mlspace/envs/vika_kurkin_clone/bin/x86_64-conda-linux-gnu-cc -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/jovyan/.mlspace/envs/vika_kurkin_clone/include -I/home/jovyan/.mlspace/envs/vika_kurkin_clone/targets/x86_64-linux/include -L/home/jovyan/.mlspace/envs/vika_kurkin_clone/targets/x86_64-linux/lib -L/home/jovyan/.mlspace/envs/vika_kurkin_clone/targets/x86_64-linux/lib/stubs -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/jovyan/.mlspace/envs/vika_kurkin_clone/include -I/home/jovyan/.mlspace/envs/vika_kurkin_clone/targets/x86_64-linux/include -L/home/jovyan/.mlspace/envs/vika_kurkin_clone/targets/x86_64-linux/lib -L/home/jovyan/.mlspace/envs/vika_kurkin_clone/targets/x86_64-linux/lib/stubs -fPIC -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/jovyan/.mlspace/envs/vika_kurk

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of the model checkpoint at /home/jovyan/shares/SR004.nfs2/chekalina/FisherKronecker/llama/llama10 were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.0.weight', 'model.layers.0.mlp.down_proj.1.weight', 'model.layers.0.mlp.gate_proj.0.weight', 'model.layers.0.mlp.gate_proj.1.weight', 'model.layers.0.mlp.up_proj.0.weight', 'model.layers.0.mlp.up_proj.1.weight', 'model.layers.0.self_attn.k_proj.0.weight', 'model.layers.0.self_attn.k_proj.1.weight', 'model.layers.0.self_attn.o_proj.0.weight', 'model.layers.0.self_attn.o_proj.1.weight', 'model.layers.0.self_attn.q_proj.0.weight', 'model.layers.0.self_attn.q_proj.1.weight', 'model.layers.0.self_attn.v_proj.0.weight', 'model.layers.0.self_attn.v_proj.1.weight', 'model.layers.1.mlp.gate_proj.0.weight', 'model.layers.1.mlp.gate_proj.1.weight', 'model.layers.1.mlp.up_proj.0.weight', 'model.layers.1.mlp.up_proj.1.weight', 'model.layers.1.self_attn.k_proj.0.weight', 'model.layers.1.self_attn.k_proj.1.weigh

In [5]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05